In [129]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [130]:
flights_df = pd.read_csv('complete_flight_info_and weather_data.csv')

# convert flight date to date object
flights_df = flights_df.drop(['date', 'date.1','CRS_DEP_TIME','ORIGIN','DEST'], axis=1)

flights_df = flights_df.dropna()
 
flights_df['FL_DATE'] = pd.to_datetime(flights_df['FL_DATE'])
del flights_df['DOT_CODE']
# Remove columns starting with 'origin' and 'dest'
columns_to_remove = [col for col in flights_df.columns if col.startswith('ORIGIN') or col.startswith('DEST')]
flights_df = flights_df.drop(columns=columns_to_remove)

# converts string TRUE/FALSE to boolean
flights_df.replace({'TRUE': True, 'FALSE': False}, inplace=True)

# convert FL_Date to year, month, day
flights_df['FL_YEAR'] = pd.to_datetime(flights_df['FL_DATE']).dt.year
flights_df['FL_MONTH'] = pd.to_datetime(flights_df['FL_DATE']).dt.month
flights_df['FL_DAY'] = pd.to_datetime(flights_df['FL_DATE']).dt.day

# drop original date time
flights_df.drop(columns=['FL_DATE'], inplace=True)

X = flights_df.loc[:, flights_df.columns != 'ARR_DELAY']
y = flights_df['ARR_DELAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 156, shuffle=True)

In [131]:
def forward_subset_selection(X, y, max_features=None, cv=5):
    selected_features = []
    best_score = float('-inf')
    
    while len(selected_features) < max_features:
        best_feature = None
        for feature in X.columns:
            if feature not in selected_features:
                features_to_try = selected_features + [feature]
                X_subset = X[features_to_try]
                model = LinearRegression()
                scores = cross_val_score(model, X_subset, y, cv=cv, scoring='r2')
                mean_score = scores.mean()
                if mean_score > best_score:
                    best_score = mean_score
                    best_feature = feature
        if best_feature is None:
            break
        selected_features.append(best_feature)
    
    return selected_features

# Example usage:
# X_train: training features, y_train: training labels
# max_features: maximum number of features to select
selected_features = forward_subset_selection(X_train, y_train, max_features=5)
print("Selected Features:", selected_features)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/suman/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/suman/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/suman/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py", line 578, in fit
    X, y = self._validate_data(
  File "/Users/suman/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/U

In [ ]:
model = tree.DecisionTreeRegressor(max_depth=2, random_state=156) 
model = model.fit(X_train, y_train)

model_text = tree.export_text(model, feature_names=list(X_train.columns))
print(model_text)

|--- dep_wind_direction_100m <= 0.59
|   |--- value: [1092.00]
|--- dep_wind_direction_100m >  0.59
|   |--- CRS_ARR_TIME <= 1000.50
|   |   |--- value: [87.46]
|   |--- CRS_ARR_TIME >  1000.50
|   |   |--- value: [71.57]



In [ ]:
fi = model.feature_importances_

names = X_train.columns
importance_dict = dict(zip(names, fi))

print("Feature Importance:")
for feature, importance in importance_dict.items():
    print(f"{feature}: {importance}")

Feature Importance:
FL_NUMBER: 0.0
CRS_ARR_TIME: 0.4726816646485719
CRS_ELAPSED_TIME: 0.0
DISTANCE: 0.0
dep_temperature_2m: 0.0
dep_apparent_temperature: 0.0
dep_rain: 0.0
dep_wind_speed_10m: 0.0
dep_wind_speed_100m: 0.0
dep_cloud_cover: 0.0
dep_cloud_cover_low: 0.0
dep_cloud_cover_mid: 0.0
dep_cloud_cover_high: 0.0
dep_wind_direction_10m: 0.0
dep_wind_direction_100m: 0.5273183353514281
dep_wind_gusts_10m: 0.0
dep_snow_depth: 0.0
dest_temperature_2m: 0.0
dest_apparent_temperature: 0.0
dest_rain: 0.0
dest_wind_speed_10m: 0.0
dest_wind_speed_100m: 0.0
dest_cloud_cover: 0.0
dest_cloud_cover_low: 0.0
dest_cloud_cover_mid: 0.0
dest_cloud_cover_high: 0.0
dest_wind_direction_10m: 0.0
dest_wind_direction_100m: 0.0
dest_wind_gusts_10m: 0.0
dest_snow_depth: 0.0
FL_YEAR: 0.0
FL_MONTH: 0.0
FL_DAY: 0.0


In [ ]:
preds = model.predict(X_test)
print(mean_squared_error(y_test, preds), r2_score(y_test, preds))

11936.168989864702 0.0025195789004965885


In [ ]:
mse = {'k':[], 'train_mse':[], 'test_mse':[]}
for k in range(1,30):
    print("Fit with max_depth:", k, end='\r', flush=True)
    
    model = tree.DecisionTreeRegressor(max_depth=k)
    model = model.fit(X_train, y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)

    mse['k'].append(k)
    mse['train_mse'].append(mean_squared_error(y_train, preds_train))
    mse['test_mse'].append(mean_squared_error(y_test, preds_test))
    
idx = mse['test_mse'].index(min(mse['test_mse']))
print('Depth of the model yielding minimum test MSE is:', mse['k'][idx])
print('Optimized model has MSE:', min(mse['test_mse']))

Depth of the model yielding minimum test MSE is: 2
Optimized model has MSE: 11936.168989864702


In [ ]:
model_rf = RandomForestRegressor (n_estimators=100, random_state=156, max_features=30)

model_rf.fit(X_train, y_train)
preds_test = model_rf.predict(X_test)

print(mean_squared_error(y_test, preds_test), r2_score(y_test, preds_test))

12919.771006462586 -0.07967796325423282


In [ ]:
from sklearn.model_selection import GridSearchCV

clf = RandomForestRegressor(random_state=156)

params = {
    'max_depth': np.arange(5,10,1),
    'n_estimators': np.arange(5,10,1)
}

grid_search = GridSearchCV(estimator=clf, param_grid=params, cv=5, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_error', return_train_score=True)
grid_search.fit(X_train, y_train)



Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=156), n_jobs=-1,
             param_grid={'max_depth': array([5, 6, 7, 8, 9]),
                         'n_estimators': array([5, 6, 7, 8, 9])},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [ ]:
# Get the best estimator and show parameters
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 8}

In [ ]:
# Evaluate the tuned model

clf_best = grid_search.best_estimator_
y_test_pred = clf_best.predict(X_test)

print('Hyperparameter tuning of random forest yields MSE and R-squared:', mean_squared_error(y_test, y_test_pred), r2_score(y_test, y_test_pred))

Hyperparameter tuning of random forest yields MSE and R-squared: 12045.09904485573 -0.0065834756235392256
